<a href="https://colab.research.google.com/github/SongThun/DataMining-HK241/blob/main/Mamba4Rec-H%26M-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir dataset
!mkdir saved

In [2]:
!cd dataset && mkdir hm
!cd ..

In [1]:
!pip install recbole torch ray
!pip install --upgrade pip setuptools wheel
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118
!pip install mamba-ssm

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0+cu121
    Uninstalling torch-2.5.0+cu121:
      Successfully uninstalled torch-2.5.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.0+cu121
    Uninstalling torchvision-0.20.0+cu121:
      Successfully uninstalled torchvision-0.20.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.0+cu121
    Uninstalling torchaudio-2.5.0+cu121:
      Successfully uninstalled torchaudio-2.5.0+cu121
  Using cached mamba_ssm-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) 

In [3]:
import pandas as pd
inter = pd.read_csv("dataset/hm/hm.inter", delimiter="\t")
inter.head()

,user_id:token,item_id:token,timestamp:float
0,0002d2ef78ec29c03a951963a2694be963c06d7123b972...,865714002,15855264.0
1,0002d2ef78ec29c03a951963a2694be963c06d7123b972...,865714002,15855264.0
2,0002d2ef78ec29c03a951963a2694be963c06d7123b972...,779250008,15855264.0
3,0002d2ef78ec29c03a951963a2694be963c06d7123b972...,836284003,15855264.0
4,0002d2ef78ec29c03a951963a2694be963c06d7123b972...,827704004,15855264.0


In [ ]:
!python run.py --config_file="config_hm.yaml" --checkpoint_file="saved/Mamba4Rec-Nov-06-2024_12-20-05.pth"

2024-11-07 15:24:43.217910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 15:24:43.457812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 15:24:43.529206: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 15:24:43.931860: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-07 15:24:45.913689: W tensorflow/comp

In [ ]:
from recbole.quick_start.quick_start import load_data_and_model
from mamba4rec import Mamba4Rec

config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='saved/Mamba4Rec-Nov-06-2024_12-20-05.pth',
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')